In [1]:
from pykrx import stock
from datetime import datetime
from util.util import *
from tqdm.notebook import tqdm 
import pickle
import numpy as np

In [2]:
start_date = '20220101'
end_date = '20231103'


market = "KOSPI"


In [3]:
################ 삼성으로 데이터 길이 동일하게 만들기 
samsung = '005930'

df = stock.get_market_ohlcv(start_date, end_date, samsung)
sample_date_len = len(df)
print(df)

################# 모든 데이터에 대해서 데이터 얻기 

               시가     고가     저가     종가       거래량       등락률
날짜                                                        
2022-01-03  79400  79800  78200  78600  13502112  0.383142
2022-01-04  78800  79200  78300  78700  12427416  0.127226
2022-01-05  78800  79000  76400  77400  25470640 -1.651842
2022-01-06  76700  77600  76600  76900  12931954 -0.645995
2022-01-07  78100  78400  77400  78300  15163757  1.820546
...           ...    ...    ...    ...       ...       ...
2023-10-30  66800  67800  66700  67300  10139270  0.000000
2023-10-31  67600  68300  66900  66900  14488892 -0.594354
2023-11-01  67500  68900  67300  68600  13775256  2.541106
2023-11-02  70000  70000  69400  69700  16350031  1.603499
2023-11-03  69700  70200  69500  69600  10275036 -0.143472

[453 rows x 6 columns]


In [4]:
tickers = stock.get_market_ticker_list(end_date, market=market)
names = [stock.get_market_ticker_name(ticker) for ticker in tickers]

print(len(tickers))

952


In [5]:
all_data = []
valid_ticker = {}


for name , ticker in tqdm(zip(names,tickers),total=len(names)):

    try : 
        df = stock.get_market_ohlcv(start_date, end_date, ticker)
    except : 
        continue
    if len(df) != sample_date_len : 
        continue
    dates = df.index
    df = rename_columns(df)
    df = add_all_feature(df)
    df_fdm = stock.get_market_fundamental(start_date, end_date, ticker, freq="d")
    df_tra_vol_all = stock.get_market_trading_volume_by_date(start_date, end_date, ticker,on='순매수')
    df_tra_vol_all = rename_to_english(df_tra_vol_all)
    merged_df_1 = df.merge(df_fdm, left_index=True, right_index=True, how='outer')
    merged_df_2 = merged_df_1.merge(df_tra_vol_all, left_index=True, right_index=True, how='outer')
    df = merged_df_2
    
    
    if len(df.columns) != 42 : 
        print(f'ticker : {ticker} , name : {name} has no 42 features.')
        continue
    
    # df = df[df.index.year == end_year]    
    df_80 = df[-80:].copy()

    
    # Check for NaN or infinite values in the dataframe
    if df_80.isnull().any().any() or np.isinf(df_80.values).any():
        print(f'ticker : {ticker} , name : {name} has Nan or Inf value.')
        continue


    dates = df_80.index    
    np_df = np.array(df_80)
    all_data.append(np_df)
    valid_ticker[ticker] = name

  0%|          | 0/952 [00:00<?, ?it/s]

ticker : 001040 , name : CJ has Nan or Inf value.
ticker : 000480 , name : CR홀딩스 has Nan or Inf value.
ticker : 012030 , name : DB has Nan or Inf value.
ticker : 365550 , name : ESR켄달스퀘어리츠 has no 42 features.
ticker : 003560 , name : IHQ has Nan or Inf value.
ticker : 001390 , name : KG케미칼 has Nan or Inf value.
ticker : 033180 , name : KH 필룩스 has Nan or Inf value.
ticker : 400760 , name : NH올원리츠 has no 42 features.
ticker : 338100 , name : NH프라임리츠 has no 42 features.
ticker : 395400 , name : SK리츠 has no 42 features.
ticker : 011810 , name : STX has Nan or Inf value.
ticker : 005725 , name : 넥센우 has Nan or Inf value.
ticker : 009190 , name : 대양금속 has Nan or Inf value.
ticker : 000300 , name : 대유플러스 has Nan or Inf value.
ticker : 001070 , name : 대한방직 has Nan or Inf value.
ticker : 004830 , name : 덕성 has Nan or Inf value.


KeyboardInterrupt: 

In [ ]:
all_data = []
valid_ticker = {}


for name , ticker in tqdm(zip(names,tickers),total=len(names)):

    try : 
        df = stock.get_market_ohlcv(start_date, end_date, ticker)
    except : 
        continue
    if len(df) != sample_date_len : 
        continue
    dates = df.index
    df = rename_columns(df)
    df = add_all_feature(df)
    df_fdm = stock.get_market_fundamental(start_date, end_date, ticker, freq="d")
    df_tra_vol_all = stock.get_market_trading_volume_by_date(start_date, end_date, ticker,on='순매수')
    df_tra_vol_all = rename_to_english(df_tra_vol_all)
    merged_df_1 = df.merge(df_fdm, left_index=True, right_index=True, how='outer')
    merged_df_2 = merged_df_1.merge(df_tra_vol_all, left_index=True, right_index=True, how='outer')
    df = merged_df_2
    
    
    if len(df.columns) != 42 : 
        print(f'ticker : {ticker} , name : {name} has no 42 features.')
        continue
    
    # df = df[df.index.year == end_year]    
    df_80 = df[-80:].copy()

    
    # Check for NaN or infinite values in the dataframe
    if df_80.isnull().any().any() or np.isinf(df_80.values).any():
        print(f'ticker : {ticker} , name : {name} has Nan or Inf value.')
        continue

    
    df_60 = df_80[:60]
    
    np_60 = np.array(df_60)
    
    input = normalize_data_per_ticker_one(np_60)
    
    
    
    
    